In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

In [2]:
from SCSCtrl import TTLServo
from jetbot import Robot
import time

Succeeded to open the port
Succeeded to change the baudrate


In [3]:
# camera setting
camera = Camera.instance(width=300, height=300)
image = widgets.Image(format='jpeg', width=300, height=300)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [4]:
idx_servo_hori = 1
idx_servo_vert1 = 2
idx_servo_vert2 = 3
idx_servo_grip = 4
idx_servo_camera = 5
speed_motor = 150

In [5]:
dir = -1

In [6]:
degree_camera = 0
degree_hori = 0
degree_vert1 = 0
degree_vert2 = 0
degree_grip = 0

In [21]:
degree_motor = 5

In [34]:
degree_slider = widgets.IntSlider(description='degree', value = degree_motor, min=0, max=30, step=1, orientation='horizontal', continuous_update=False)

In [40]:
def update_degree(change):
    global degree_motor
    degree_motor = change.new
    print(f'Updated value: {degree_motor}')  # 변수 값 업데이트 시 출력

degree_slider.observe(update_degree, names='value')

In [42]:
print(degree_motor)

12


In [8]:
def check_in_range(degree):
    if degree > 150:
        return 150
    if degree < -150:
        return -150
    return degree

In [73]:
def control_horizontal(degree):
    global idx_servo_hori, speed_motor
    global degree_hori, dir
    degree_hori = check_in_range(degree + degree_hori)
    TTLServo.servoAngleCtrl(idx_servo_hori, degree_hori, dir, speed_motor)

def control_vertical1(degree):
    global idx_servo_vert1, speed_motor
    global degree_vert1, dir
    degree_vert1 = check_in_range(degree + degree_vert1)
    TTLServo.servoAngleCtrl(idx_servo_vert1, degree_vert1, dir, speed_motor)
    
def control_vertical2(degree):
    global idx_servo_vert2, speed_motor
    global degree_vert2, dir
    degree_vert2 = check_in_range(degree + degree_vert2)
    TTLServo.servoAngleCtrl(idx_servo_vert2, degree_vert2, dir, speed_motor)
    
def control_grip(degree):
    global idx_servo_grip, speed_motor
    global degree_grip, dir
    degree_grip = check_in_range(degree + degree_grip)
    TTLServo.servoAngleCtrl(idx_servo_grip, degree_grip, dir, speed_motor)

def control_camera(degree):
    global idx_servo_camera, speed_motor
    global degree_camera, dir
    degree_camera = check_in_range(degree + degree_camera)
    print(degree_camera)
    TTLServo.servoAngleCtrl(idx_servo_camera, degree_camera, dir, speed_motor)

In [79]:
# @type is whether staying grip or release
def init(type=True):
    global idx_servo_hori, idx_servo_vert1, idx_servo_vert2, idx_servo_grip, idx_servo_camera, speed_motor
    global degree_hori, degree_vert1, degree_vert2, degree_grip, degree_camera, dir
    degree_hori = -8
    degree_vert1 = 0
    degree_vert2 = 0
    degree_grip = -8
    degree_camera = 0
    TTLServo.servoAngleCtrl(idx_servo_hori, degree_hori, dir, speed_motor)
    TTLServo.servoAngleCtrl(idx_servo_vert1, degree_vert1, dir, speed_motor)
    TTLServo.servoAngleCtrl(idx_servo_vert2, degree_vert2, dir, speed_motor)
    if type:
        TTLServo.servoAngleCtrl(idx_servo_grip, degree_grip, dir, speed_motor)
    TTLServo.servoAngleCtrl(idx_servo_camera, degree_camera, dir, speed_motor)

In [80]:
init()

In [12]:
PT_layout = widgets.Layout(width='100px', height='80px', align_self='center')

In [39]:
cameraUp_button = widgets.Button(description='LookUp', button_style='success', layout=PT_layout)
cameraDown_button = widgets.Button(description='LookDown', button_style='success', layout=PT_layout)
cameraUp_button.on_click(lambda x: control_camera(degree_motor))
cameraDown_button.on_click(lambda x: control_camera(-1 * degree_motor))

bodyLeft_button = widgets.Button(description='LookLeft', button_style='success', layout=PT_layout)
bodyRight_button = widgets.Button(description='LookRight', button_style='success', layout=PT_layout)
bodyLeft_button.on_click(lambda x: control_horizontal(degree_motor))
bodyRight_button.on_click(lambda x: control_horizontal(-1 * degree_motor))

grap_button = widgets.Button(description='Grap', button_style='success', layout=PT_layout)
Release_button = widgets.Button(description='Release', button_style='success', layout=PT_layout)
grap_button.on_click(lambda x: control_grip(degree_motor))
Release_button.on_click(lambda x: control_grip(-1 * degree_motor))

vert1Up_button = widgets.Button(description='Vert1 Up', button_style='success', layout=PT_layout)
vert1Down_button = widgets.Button(description='Vert1 Down', button_style='success', layout=PT_layout)
vert1Up_button.on_click(lambda x: control_vertical1(degree_motor))
vert1Down_button.on_click(lambda x: control_vertical1(-1 * degree_motor))

vert2Up_button = widgets.Button(description='Vert2 Up', button_style='success', layout=PT_layout)
vert2Down_button = widgets.Button(description='Vert2 Down', button_style='success', layout=PT_layout)
vert2Up_button.on_click(lambda x: control_vertical2(-1 * degree_motor))
vert2Down_button.on_click(lambda x: control_vertical2(degree_motor))

topBtn_box = widgets.HBox([cameraUp_button, bodyLeft_button, grap_button, vert1Up_button, vert2Up_button], layout=widgets.Layout(align_self='center'))
bottomBtn_box = widgets.HBox([cameraDown_button, bodyRight_button, Release_button, vert1Down_button, vert2Down_button], layout=widgets.Layout(align_self='center'))
surPT_box = widgets.VBox([topBtn_box, bottomBtn_box, degree_slider])

display(surPT_box)

In [14]:
robot = Robot()

In [38]:
def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.4)
    time.sleep(0.5)
    robot.stop()

def step_backward(change):
    robot.backward(0.4)
    time.sleep(0.5)
    robot.stop()

def step_left(change):
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

def step_right(change):
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()

In [37]:
# define move buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

In [82]:
print(degree_camera)
print(degree_hori) 
print(degree_vert1)
print(degree_vert2)
print(degree_grip) 

0
-8
0
0
-8


In [87]:
init()

In [85]:
control_vertical1(-73)
control_vertical2(-44)

In [86]:
while True:
    if degree_vert1 == -73 and degree_vert2 == -44:
        control_grip(27)
        init(False)
        break